In [1]:
import arimax as model
import pandas as pd
import numpy as np
import statistics as s
import scipy.optimize as opt

In [2]:
df=pd.read_csv("Data1997-2022.csv")
df['Intercept'] = np.ones(len(df))
df

,Unnamed: 0,CPI,Monthly salary,Policy rate,Unemployed,Intercept
0,0,69.3,20126,3.50,4.0,1.0
1,1,69.6,20126,3.50,3.7,1.0
2,2,69.7,20126,3.50,3.6,1.0
3,3,69.7,20126,3.50,3.4,1.0
4,4,69.8,20126,3.50,3.2,1.0
...,...,...,...,...,...,...
307,307,123.9,53150,1.75,1.6,1.0
308,308,125.6,53150,1.75,1.6,1.0
309,309,126.0,53150,2.25,1.6,1.0
310,310,125.8,53150,2.50,1.6,1.0


In [3]:
cpi_diff12 = df['CPI'].diff(12)
exog_data = df[['Intercept', 'Monthly salary', 'Policy rate', 'Unemployed']]

cpi_diff12


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
307    7.6
308    8.1
309    8.8
310    7.7
311    7.0
Name: CPI, Length: 312, dtype: float64

In [5]:
ARX = model.ARMAX(2, 0, cpi_diff12[12:], exog_data[12:])
evo_init = -0.5 *np.ones(2) #np.array([-1.6242, -0.8331])
var_init = 50 #72
beta_init = np.zeros(4) #np.array([1.101e-08, -0.0001, 0.0251, 0.0449])

op = ARX.fit_kalman(evo_init, var_init, beta_init)

In [6]:
op.x

array([ 1.28093339e-01, -8.84194514e-01,  1.98032787e+00, -2.61005457e+00,
        1.40646219e-04,  3.11146065e-01, -4.59944484e-01])

In [7]:
def get_jacobian(params):
    return opt.approx_fprime(params, ARX.kalman_log_likelihood)

def get_hessian(params):
    return opt.approx_fprime(params, get_jacobian)

get_hessian(op.x)

array([[ 4.09600000e+03,  1.02400000e+04,  1.53600000e+03,
         6.14400000e+03,  4.24550400e+06,  3.58400000e+03,
         3.58400000e+03],
       [ 1.02400000e+04,  5.47840000e+04,  7.68000000e+03,
         7.16800000e+03,  1.55827200e+07,  4.60800000e+03,
         5.63200000e+03],
       [ 1.53600000e+03,  7.68000000e+03, -5.12000000e+02,
         3.07200000e+03,  2.13350400e+06,  5.12000000e+02,
         0.00000000e+00],
       [ 6.14400000e+03,  7.16800000e+03,  3.07200000e+03,
         7.16800000e+03,  2.71303680e+07,  4.60800000e+03,
         7.16800000e+03],
       [ 4.24550400e+06,  1.55827200e+07,  2.13350400e+06,
         2.71303680e+07,  1.05022644e+12,  6.08163840e+07,
         7.63755520e+07],
       [ 3.58400000e+03,  4.60800000e+03,  5.12000000e+02,
         4.60800000e+03,  6.08163840e+07,  8.70400000e+03,
         6.65600000e+03],
       [ 3.58400000e+03,  5.63200000e+03,  0.00000000e+00,
         7.16800000e+03,  7.63755520e+07,  6.65600000e+03,
         6.6560000

In [6]:
ARX.summary()

                                    Results                                     
Dep. Variable:                                  CPI
Model:                              ARIMAX( 2 , 0 )
No. Observations:                               300
AIC                                        AICvalue
BIC                                        BICvalue
Log Likelihood                             Logvalue
                     coef    std.err          z      P>|z|     [0.025     0.975]
--------------------------------------------------------------------------------
Intercept       4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
Monthly salary  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
Policy rate     4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
Unemployed      4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
ar.L1           4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
ar.L2           4.236e+00  4.236e+00  4.